In [ ]:
import numpy as np
import pandas as pd

import datetime
from data_collection.data_collection import Logger
import plotly.express as px
import scipy.signal as ss

import torchaudio as ta
import torch as tch
tafn = ta.functional
from IPython.display import Audio
from nb_tools import *

%load_ext autoreload
%autoreload 2

In [ ]:
from functools import partial

rate=16000
Audio = partial(Audio, rate=16000)


In [ ]:
sesspath = '../log/testsession2024-07-06 20:56:00.157529/'
l = Logger(sesspath,name='PitchAngularVelocityController', overwrite_ok=True)
lw2 = Logger(sesspath,name='TwoWheelsV3', overwrite_ok=True)
df = l.load_as_df()
dfw2 = lw2.load_as_df()
df['power_norm'] = df['power']/1e4

In [ ]:
from scipy.fft import fft, fftfreq
v = df['sig'].dropna()
v = v.apply(fft)
v = v.apply(lambda x: x.real**2+x.imag**2)
v = v.apply(lambda x: x[:len(x)//2])
freqs = fftfreq((4000), 1/16000)[:2000]
spectra = np.log(np.stack(v.tolist())).T[::-1]
px.imshow(spectra, y=freqs[::-1], aspect='auto')

In [ ]:
from typing import *
from nb_tools import show_global_variables

from scipy.fft import fft, fftfreq

@show_global_variables
def show_spectrum(sig, fs=16000):
    return px.line(x=fftfreq(len(sig), 1/fs)[:len(sig)//2], y=np.log(np.abs(fft(sig)**2+1e-100)[:len(sig)//2]))

@show_global_variables
def piece_signal(sig1, sig2, overlap_length=50):
    step=0
    while True:
        #print((-step-overlap_length))
        #print((len(sig1)-step))
        sig_to_test = sig1[(-step-overlap_length):(len(sig1)-step)]
        
        if (sig2[:overlap_length] == sig_to_test ).all():
            return np.r_[sig1[:(-step-overlap_length)], sig2]
        else:
            step+=1 
        if step > len(sig1):
            break
    


@show_global_variables
def piece_signals(sigs: List[np.ndarray]):

    full_sig = sigs[0]
    for i in range(len(sigs)-1):
        try: 
            full_sig = piece_signal(full_sig, sigs[i+1])
        except:
            return full_sig
    return full_sig
#sigs = piece_signals(df['sig'].tolist())

In [ ]:
fs=16000
class IIRPeakFilter:
    @show_global_variables
    def __init__(self, freq, q, fs):
        self.freq = freq
        self.q = q
        self.fs = fs
        self.b, self.a = ss.iirpeak(freq, q, fs)

    @show_global_variables
    def plot(self):
        freq, h = ss.freqz(self.b, self.a, fs=self.fs)
        return px.line(
            y=20*np.log10(np.maximum(np.abs(h),1e-5)), 
            x=freq
        )
    
    def apply(self, x):
        return ss.lfilter(self.b,self.a,x)

In [ ]:
f0 = IIRPeakFilter(270, 3, fs)
f1 = IIRPeakFilter(2300, 23, fs)
f2 = IIRPeakFilter(3000, 30, fs)

In [ ]:
ts = np.cumsum(np.random.random(fs*3)/(fs)*(1)+(1-(0.5)))/fs
x=ss.square(2*np.pi*170*ts)
px.line(x)

In [ ]:
show_spectrum(x)

In [ ]:
r = f0.apply(f1.apply(f2.apply(x)))
show_spectrum(r)

In [ ]:
rv2 = f0.apply(x)*5+f1.apply(x)*10+f2.apply(x)*2
show_spectrum(rv2)

In [ ]:
Audio(rv2)

In [ ]:
# slowly vary the pitch the formant frequencies? 

In [ ]:
px.line(rv2)

In [ ]:
Audio(r)

In [ ]:
show_spectrum(sig_f[16000*2:16000*5])

In [ ]:
sig_f = butterworth_filter(sigs, 16000, 100,1500)
Audio(sig_f, rate=16000)

In [ ]:
from functools import partial
@show_global_variables
def freq_phase(n, fs, freq):
    return np.linspace(0, n/fs*freq*np.pi*2, n, endpoint=False)%(2*np.pi)

@show_global_variables
def freq_phase_harmonics(f0, nth_har: List, n, fs):
    result = []

    for _nth_bar in nth_har:
        result.append(freq_phase(n, fs, f0*(_nth_bar+1)))

    return np.stack(result, axis=0)




fs = 16000
n = 32000
syn_sig = np.sin(freq_phase_harmonics(120, np.arange(7), n, fs))*np.array([70,99,70,1,1,1,1])[..., None]
#Audio(syn_sig.mean(0), rate=fs)
show_spectrum(syn_sig.mean(0))


In [ ]:
Audio(syn_sig.mean(0), rate=fs)

In [ ]:
120*(np.arange(20)+1)

In [ ]:
px.line(np.array([]))

In [ ]:
px.line(syn_sig.mean(0))

In [ ]:
show_spectrum(syn_sig.mean(0))

In [ ]:
px.line(sig_f[16000*2:16000*5])

In [ ]:
Audio(sig_f[16000*3:16000*4], rate=16000)